In [ ]:
#conda install -n grpc ipykernel --update-deps --force-reinstall

In [2]:
! python -m pip install grpcio
! pip install grpcio-tools

     |████████████████████████████████| 2.1 MB 2.2 MB/s eta 0:00:01
     |████████████████████████████████| 960 kB 26.7 MB/s eta 0:00:01


# Proto file

In [7]:
%%writefile ../protos/chatbot.proto
syntax = "proto3";

package chatPack;

service ChatService {
  rpc ChatFunc (ChatRequest) returns (ChatReply) {}
  rpc ChatStream (stream StreamText) returns (stream StreamText) {}
}

message ChatRequest {
  string name = 1;
}
message ChatReply {
  string result = 1;
}

message StreamText {
  string name = 1;
  string result = 2;
}


Overwriting ../protos/chatbot.proto


In [8]:
! python -m grpc_tools.protoc -I ../protos/ --python_out=. --grpc_python_out=. chatbot.proto


# Server

In [9]:
%%writefile server.py
from concurrent import futures
import logging

import grpc
import chatbot_pb2
import chatbot_pb2_grpc


class ChatService(chatbot_pb2_grpc.ChatService):

    def ChatFunc(self, request, context):
        return chatbot_pb2.ChatReply(result='Hello, %s!' % request.name)
    def ChatStream(self, request_iterator, context):
        for new_data in request_iterator:
            print('ChatStream Recive data:'+new_data.name+':'+new_data.result)
            yield chatbot_pb2.StreamText(name='1',result=new_data.result+' ok')
        print('End ChatStream.')

def serve():
    server = grpc.server(futures.ThreadPoolExecutor(max_workers=10))
    chatbot_pb2_grpc.add_ChatServiceServicer_to_server(ChatService(), server)
    server.add_insecure_port('[::]:8099')
    server.start()
    server.wait_for_termination()


if __name__ == '__main__':
    logging.basicConfig()
    serve()

Overwriting server.py


# Client

In [6]:
from __future__ import print_function
import logging
import grpc
import chatbot_pb2
import chatbot_pb2_grpc

def run():
    with grpc.insecure_channel('localhost:8099') as channel:
        stub = chatbot_pb2_grpc.ChatServiceStub(channel)
        response = stub.ChatFunc(chatbot_pb2.ChatRequest(name='blue'))
    print("Chat client received: " + response.result)

logging.basicConfig()
run()


Chat client received: Hello blue


# async client

In [6]:
! pip install asyncio

     |████████████████████████████████| 101 kB 2.0 MB/s ta 0:00:01


In [5]:
%%writefile async_client.py
import asyncio
import logging

import grpc
import chatbot_pb2
import chatbot_pb2_grpc


async def runAsync() -> None:
    async with grpc.aio.insecure_channel('localhost:8099') as channel:
        stub = chatbot_pb2_grpc.ChatServiceStub(channel)
        response = await stub.ChatFunc(chatbot_pb2.ChatRequest(name='you'))
    print("Chat client received: " + response.result)


if __name__ == '__main__':
    logging.basicConfig()
    # asyncio.run(run())
    loop = asyncio.get_event_loop()
    loop.run_until_complete(runAsync())

Overwriting async_client.py


In [6]:
! python async_client.py

Chat client received: Hello, you!


# Client stream

In [4]:
import grpc
import chatbot_pb2
import chatbot_pb2_grpc
def make_data(name, result):
    return chatbot_pb2.StreamText(
        name=name,
        result=result)
def generate_messages():
    messages = [
        make_data('0','zero'),
        make_data('0','one'),
        make_data('0','two'),
    ]
    for msg in messages:
        yield msg
def biStreamClient(stub): 
    response = stub.ChatStream(generate_messages())
    for resp in response:
        print("biStream client received: " + str(resp.name) + ':' + resp.result)
def run():
    with grpc.insecure_channel('localhost:8099') as channel:
        stub = chatbot_pb2_grpc.ChatServiceStub(channel)
        biStreamClient(stub)
run()

biStream client received: 1:zero ok
biStream client received: 1:one ok
biStream client received: 1:two ok
